<a href="https://colab.research.google.com/github/Tech-Virtuoso/AI_assistant/blob/main/CNN_Plant_dieases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
train_dir= "/content/drive/MyDrive/Datasets/Plant dieases/Test/Test"
test_dir="/content/drive/MyDrive/Datasets/Plant dieases/Train/Train"
val_dir="/content/drive/MyDrive/Datasets/Plant dieases/Validation/Validation"

In [68]:
import tensorflow as tf

In [69]:
# Creating a Dataset for the Training data
train = tf.keras.utils.image_dataset_from_directory(
    train_dir,  # Directory where the Training images are located
    labels = 'inferred', # Classes will be inferred according to the structure of the directory
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,    # Number of processed samples before updating the model's weights
    image_size = (256, 256), # Defining a fixed dimension for all images
    shuffle = True,  # Shuffling data
    seed = 42,  # Random seed for shuffling and transformations
    validation_split = 0, # We don't need to create a validation set from the training set
    crop_to_aspect_ratio = True # Resize images without aspect ratio distortion
)

Found 150 files belonging to 3 classes.


In [70]:
# Creating a dataset for the Test data
test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 1322 files belonging to 3 classes.


In [71]:
# Creating a dataset for the Test data
validation = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 60 files belonging to 3 classes.


In [72]:
print('\nTraining Dataset:', train)
print('\nTesting Dataset:', test)
print('\nValidation Dataset:', validation)


Training Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

Testing Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

Validation Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>


In [73]:


scaler = tf.keras.layers.Rescaling(1./255) # Defining scaler values between 0 to 1
# Rescaling datasets
train = train.map(lambda x, y: (scaler(x), y))
test = test.map(lambda x, y: (scaler(x), y))
validation = validation.map(lambda x, y: (scaler(x), y))

In [74]:
seed=42
# Creating data augmentation pipeline
augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(
        factor = (-.25, .3),
        fill_mode = 'reflect',
        interpolation = 'bilinear',
        seed = seed),


        tf.keras.layers.RandomBrightness(
        factor = (-.45, .45),
        value_range = (0.0, 1.0),
        seed = seed),

        tf.keras.layers.RandomContrast(
        factor = (.5),
        seed = seed)
    ]
)

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

# Initiating model without strategy.scope()
model = Sequential()

# Feature Learning Layers
model.add(Conv2D(32,                  # Number of filters/Kernels
                 (3,3),               # Size of kernels (3x3 matrix)
                 strides = 1,         # Step size for sliding the kernel across the input (1 pixel at a time).
                 padding = 'same',    # 'Same' ensures that the output feature map has the same dimensions as the input by padding zeros around the input.
                input_shape = (256,256,3) # Input image shape
                ))

model.add(augmentation)  # Assuming you defined an augmentation layer earlier
model.add(Activation('relu'))  # Activation function
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

model.add(Conv2D(256, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

# Flattening tensors
model.add(Flatten())

# Fully-Connected Layers
model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(3, activation='softmax'))  # Classification layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [76]:
# Compiling model

from keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate=0.001)  # Or whatever optimizer you're using
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [83]:


# Defining an Early Stopping and Model Checkpoints
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',          # Metric to monitor (e.g., validation loss)
    mode='max',                 # Mode to monitor ('min' for loss, 'max' for accuracy)
    restore_best_weights=True    # Restore model weights from the epoch with the best value of the monitored metric
)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',     # Path where the model file will be saved
    monitor='val_accuracy',           # Metric to monitor (e.g., validation loss)
    save_best_only=True,           # Save only the best model based on the monitored                      # Verbosity mode (1 for progress messages)
)


In [86]:
history = model.fit(
    train,                           # Training dataset
    validation_data=validation,      # Validation dataset
    epochs=100,                       # Number of epochs
    batch_size=10,                   # Batch size
)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 816ms/step - accuracy: 0.3922 - loss: 48.9163 - val_accuracy: 0.3333 - val_loss: 9.4047
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4102 - loss: 31.2708 - val_accuracy: 0.3333 - val_loss: 21.8100
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.3837 - loss: 46.1624 - val_accuracy: 0.4667 - val_loss: 4.5250
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 836ms/step - accuracy: 0.4537 - loss: 55.4344 - val_accuracy: 0.3500 - val_loss: 35.8213
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 837ms/step - accuracy: 0.4821 - loss: 26.7073 - val_accuracy: 0.3500 - val_loss: 23.9915
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 944ms/step - accuracy: 0.4596 - loss: 39.9989 - val_accuracy: 0.3333 - val_loss: 115.6396
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 986ms/step - accuracy: 0.4558 - loss: 35.0844 - val_accuracy: 0.3333 - val_loss: 57.5871
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 874ms/step - accuracy: 0.4719 - loss: 27.88

In [89]:
# Loading best weights
model.load_weights('best_model.keras')

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.keras'